# Introdução
___
Este arquivo tem por objetivo realizar o tratamento de dados do arquivo `cs_bisnode_panel.csv`.

### Objetivos do tratamento de dados:
* Remova as colunas ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D']  pois elas apresentam um percentual considerável de missing data ✅
* Remova de seus dados os registros do ano de 2016 ✅
* Criar uma coluna para variavel resposta (use o conceito de que uma empresa deixou de operar se ela esteve ativa no
ano X, mas não apresentou vendas em X + 2 anos) `# trabalhar nisso`
* Filtre para trabalhar apenas com empresas do ano de 2012 ✅
* usar np.where para ajustar Sales < 0 você já pode substituir por 0 ✅
* * Criar uma nova coluna para a escala logaritima de `Sales`✅
* Essa variável (`Sales`) é bastante assimétrica, concorda? Será que vale criar novas
colunas que representem o valor em log  dessa coluna?✅
* Será que isso também se aplica para as demais? `checar isso durante o loop de retreinamento`
* Crie novas colunas, como idade da empresa (faça isso pela subtração de
founded_year  e year ). Ah, cuide bem dos missing values. np.where pode ajudar
bastante!
* Filtre seus dados para ter empresas que possuem receita (revenue) abaixo de 10
milhões de euros e acima de 1000 euros
* Busque sempre embasar qualquer decisão de tratamento das variáveis. Faça isso
com o auxílio de estatísticas descritivas e também de gráficos de apoio.


# Importação dos dados e tratamento

In [1]:
import pandas as pd
import numpy as np

In [2]:
#pip install missingno

In [3]:
import missingno as msno

In [4]:
df = pd.read_csv("cs_bisnode_panel.csv")
df.head()

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1001034.0,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1001034.0,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1001034.0,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1001034.0,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1001034.0,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333


In [5]:
#msno.matrix(df)

## retirnado as colunas `'COGS', 'finished_prod', 'net_dom_sales','net_exp_sales', 'wages', 'D'` devido á alta taxa de missing
___

Retirando todas as colunas com alta taxa de missing, de acorda com um limite pré-definido:

In [6]:
#Checando as colunas e seus respectivos valores de missing
percent_missing = df.isnull().mean() * 100
missing_columns = percent_missing[percent_missing > 0].sort_values(ascending=False)
print(missing_columns)

D                    100.000000
finished_prod         93.925213
wages                 93.752193
COGS                  93.656998
net_dom_sales         93.656998
net_exp_sales         93.656998
exit_year             86.499276
exit_date             80.481466
labor_avg             50.909394
birth_year            38.848761
founded_year          19.614771
ceo_count             19.604348
origin                19.604348
gender                19.604348
inoffice_days         19.604348
female                19.604348
foreign               19.604348
extra_inc              6.437503
extra_exp              6.437503
extra_profit_loss      5.976813
profit_loss_year       3.433983
ind                    3.394029
material_exp           2.793325
personnel_exp          2.793325
amort                  2.793325
inc_bef_tax            2.583826
sales                  2.583826
tang_assets            0.510720
nace_main              0.360631
ind2                   0.360631
region_m               0.291840
intang_a

Retirando as colunas com valor de missing acima do limite exceto a coluna `founded_year` que será necessária adiante

In [7]:
limite = 0.1  # limite da taxa de missing

# Excluindo as colunas acima do limite de missing, exceto 'founded_year'
colunas_para_manter = df.columns[df.isnull().mean() <= limite].tolist()

# Garantindo que a coluna 'founded_year' esteja na lista, mesmo que tenha mais de 10% de missing
if 'founded_year' not in colunas_para_manter:
    colunas_para_manter.append('founded_year')

# Criando o DataFrame filtrado
df_limpo = df[colunas_para_manter]

# Verificando o DataFrame resultante
print(df_limpo)


             comp_id       begin         end       amort   curr_assets  \
0       1.001034e+06  2005-01-01  2005-12-31  692.592590   7266.666504   
1       1.001034e+06  2006-01-01  2006-12-31  603.703674  13122.222656   
2       1.001034e+06  2007-01-01  2007-12-31  425.925934   8196.295898   
3       1.001034e+06  2008-01-01  2008-12-31  300.000000   8485.185547   
4       1.001034e+06  2009-01-01  2009-12-31  207.407410   5137.037109   
...              ...         ...         ...         ...           ...   
287824  4.641209e+11  2011-01-01  2011-12-31    0.000000   1807.407349   
287825  4.641209e+11  2012-01-01  2012-12-31    0.000000   1518.518555   
287826  4.641209e+11  2013-01-01  2013-12-31    0.000000    988.888916   
287827  4.641209e+11  2014-01-01  2014-12-31    0.000000    644.444458   
287828  4.641209e+11  2015-01-01  2015-12-31    0.000000    166.666672   

           curr_liab  extra_exp  extra_inc  extra_profit_loss  fixed_assets  \
0        7574.074219        0.0 

Como a base dae dados possuí um número alto de colunas, fizemos um filtro mais rigoroso para missing, uma vez que isso não afetará o numero de variavéis para treinamento do modelo de forma muito significativa, e nos garantirá um modelo melhor por ser treinado em um banco de dados com baixa taxa de missing.

In [8]:
# Checando a taxa de missing nas colunas após filtragem
percent_missing = df_limpo.isnull().mean() * 100
missing_columns = percent_missing[percent_missing > 0].sort_values(ascending=False)
print(missing_columns)

founded_year         19.614771
extra_exp             6.437503
extra_inc             6.437503
extra_profit_loss     5.976813
profit_loss_year      3.433983
ind                   3.394029
material_exp          2.793325
amort                 2.793325
personnel_exp         2.793325
inc_bef_tax           2.583826
sales                 2.583826
tang_assets           0.510720
nace_main             0.360631
ind2                  0.360631
region_m              0.291840
intang_assets         0.048640
inventories           0.045513
liq_assets            0.045513
curr_assets           0.045513
fixed_assets          0.045513
share_eq              0.045513
subscribed_cap        0.045513
curr_liab             0.045513
founded_date          0.017719
dtype: float64


In [9]:
# checando as colunas presentas no dataframe para ver quantas variaveis teremos para trabalhar em nosso modelo
df_limpo.shape[1]

32

## Removendo dados do ano de 2016
___

### convertendo as colunas para o formato datetime: 

In [10]:
df_limpo["begin"] = pd.to_datetime(df_limpo['begin'])
df_limpo["end"] = pd.to_datetime(df_limpo['end'])

C:\Users\helio\AppData\Local\Temp\ipykernel_27404\11934650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo["begin"] = pd.to_datetime(df_limpo['begin'])
C:\Users\helio\AppData\Local\Temp\ipykernel_27404\11934650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo["end"] = pd.to_datetime(df_limpo['end'])


In [11]:
# retinrando os dados em que o ano seja 2016 ou maior
df_limpo = df_limpo[df_limpo["begin"] < "2016"]
df_limpo.head()

,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,fixed_assets,...,balsheet_length,balsheet_notfullyear,year,nace_main,ind2,ind,urban_m,region_m,founded_date,founded_year
0,1001034.0,2005-01-01,2005-12-31,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,1229.629639,...,364,0,2005,5630.0,56.0,3.0,1,Central,1990-11-19,1990.0
1,1001034.0,2006-01-01,2006-12-31,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,725.925903,...,364,0,2006,5630.0,56.0,3.0,1,Central,1990-11-19,1990.0
2,1001034.0,2007-01-01,2007-12-31,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,1322.222168,...,364,0,2007,5630.0,56.0,3.0,1,Central,1990-11-19,1990.0
3,1001034.0,2008-01-01,2008-12-31,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,1022.222229,...,365,0,2008,5630.0,56.0,3.0,1,Central,1990-11-19,1990.0
4,1001034.0,2009-01-01,2009-12-31,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,814.814819,...,364,0,2009,5630.0,56.0,3.0,1,Central,1990-11-19,1990.0


### Criando a coluna indicadora de operação da empresa 
(se a empresa está operante em x+2 anos ou Não)

In [12]:
df_sorted = df_limpo.sort_values(by=['comp_id', 'begin'])

df_limpo['operates_within_2_years'] = df_limpo.groupby('comp_id')['sales'].shift(-2).apply(lambda x: 1 if x > 0 else 0)

df_limpo[["comp_id","begin","end","sales","operates_within_2_years"]].head(30) # para ver se o codigo funcionou


,comp_id,begin,end,sales,operates_within_2_years
0,1001034.0,2005-01-01,2005-12-31,6.275185e+04,1
1,1001034.0,2006-01-01,2006-12-31,6.462593e+04,1
2,1001034.0,2007-01-01,2007-12-31,6.510000e+04,1
3,1001034.0,2008-01-01,2008-12-31,7.808519e+04,1
4,1001034.0,2009-01-01,2009-12-31,4.538889e+04,0
5,1001034.0,2010-01-01,2010-12-31,9.929630e+03,0
6,1001034.0,2011-01-01,2011-12-31,0.000000e+00,0
7,1001034.0,2012-01-01,2012-12-31,0.000000e+00,0
8,1001034.0,2013-01-01,2013-12-31,0.000000e+00,0
9,1001034.0,2014-01-01,2014-12-31,0.000000e+00,0


* `groupby('comp_id')`: Agrupa os dados por empresa usando comp_id para que o cálculo de sales dois anos à frente seja feito separadamente para cada empresa.
* `.shift(-2)`: Pega o valor de sales duas linhas à frente dentro de cada grupo, o que equivale a dois anos à frente, devido à ordenação das datas.
* `apply(lambda x: 1 if x > 0 else 0)`: Define 1 se o valor for positivo e 0 caso contrário.

## Trabalhando as incosistencias
___

### Ajustando a coluna sales

### Filtrando os dados para empresas com revenue abaixo de 10 milhões
Como não há uma coluna `revenue`, iremos considerar que `sales` corresponde à receita da empresa

In [13]:
df_limpo['sales'].describe()

count    2.706770e+05
mean     4.888433e+05
std      3.874202e+06
min     -1.472559e+07
25%      4.685185e+03
50%      3.029259e+04
75%      1.109593e+05
max      1.110294e+08
Name: sales, dtype: float64

In [14]:
df_limpo['sales'] = np.where(df_limpo['sales'] < 0, 0, df_limpo['sales']) # retirando valores negativos
df_limpo = df_limpo[(df_limpo['sales'] > 1000) & (df_limpo['sales'] < 10_000_000)] # filtrando os dados

In [15]:
df_limpo['sales'].describe()

count    2.182860e+05
mean     2.473741e+05
std      7.867271e+05
min      1.003704e+03
25%      1.640000e+04
50%      4.687593e+04
75%      1.428389e+05
max      9.997007e+06
Name: sales, dtype: float64

## Filtro para trabalhar apenas com empresas do ano de 2012
___

In [16]:
empresas_2012 = df_limpo[df_limpo['begin'] == "2012"]

In [17]:
empresas_2012.head()

,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,fixed_assets,...,balsheet_notfullyear,year,nace_main,ind2,ind,urban_m,region_m,founded_date,founded_year,operates_within_2_years
14,1001541.0,2012-01-01,2012-12-31,481.481476,9629.629883,1303.703735,0.0,0.000000,0.000000,190566.671875,...,0,2012,5610.0,56.0,3.0,3,Central,2008-02-24,2008.0,1
23,1002029.0,2012-01-01,2012-12-31,14929.629883,203885.187500,120444.453125,0.0,0.000000,0.000000,23459.259766,...,0,2012,2711.0,27.0,2.0,3,East,2006-07-03,2006.0,1
35,1003200.0,2012-01-01,2012-12-31,25.925926,22.222221,10996.295898,0.0,0.000000,0.000000,0.000000,...,0,2012,5630.0,56.0,3.0,1,Central,2003-10-21,2003.0,0
56,1011889.0,2012-01-01,2012-12-31,36625.925781,160166.671875,18911.111328,0.0,0.000000,0.000000,933574.062500,...,0,2012,5510.0,55.0,3.0,2,West,1992-11-09,1992.0,1
68,1014183.0,2012-01-01,2012-12-31,12551.851562,199903.703125,8274.074219,0.0,7.407407,7.407407,118229.632812,...,0,2012,5510.0,55.0,3.0,2,Central,2001-12-21,2001.0,1


### Checando a assimetria da coluna `Sales`

In [18]:
empresas_2012["sales"].skew()

7.21070047055794

Como a skewness esta acima de zero (bem acima) temos que a coluna `sales` possuí alta assímetria positiva

Criando uma nova coluna para a escala logaritima de `sales`

In [19]:
empresas_2012["sales"] = np.log1p(empresas_2012['sales'])

C:\Users\helio\AppData\Local\Temp\ipykernel_27404\3882735515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_2012["sales"] = np.log1p(empresas_2012['sales'])


In [20]:
empresas_2012["sales"].skew()

0.28412661200616257

Agora temos que a escala logaritima da coluna `sales_log` possue sua assimetria bem mais próxima de zero em relação à `sales`, indicando uma alta redução na assimetria

Checando a assimetria de outras colunas do dataset

In [21]:
numeric_cols = empresas_2012.select_dtypes(include=[np.number])

skew_values = numeric_cols.skew()

print(skew_values.sort_values(ascending = False))

extra_exp                  141.373302
extra_inc                  129.229586
extra_profit_loss           69.127878
tang_assets                 49.023178
intang_assets               46.847875
curr_liab                   46.670760
fixed_assets                43.586954
inventories                 39.431871
subscribed_cap              35.054083
amort                       27.087272
liq_assets                  20.841177
share_eq                    17.386082
curr_assets                 15.375569
balsheet_notfullyear        14.050990
personnel_exp               12.146850
material_exp                 8.087935
comp_id                      0.662560
sales                        0.284127
balsheet_flag                0.000000
year                         0.000000
urban_m                     -0.151931
founded_year                -0.414074
nace_main                   -0.699281
ind2                        -0.704084
ind                         -1.255685
operates_within_2_years     -1.523076
balsheet_len

### Criando a coluna `Idade_da_empresa`

In [22]:
empresas_2012['Idade_da_empresa'] = np.where(
    (empresas_2012['year'].isna()) | (empresas_2012['founded_year'].isna()),  # Condição: se 'year' ou 'founded_year' forem NaN
    np.nan,  # Se a condição for verdadeira (há missing values), preenche com NaN
    empresas_2012['year'] - empresas_2012['founded_year']  # Caso contrário, realiza a subtração normalmente
)
empresas_2012['Idade_da_empresa'].head()

C:\Users\helio\AppData\Local\Temp\ipykernel_27404\793854455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_2012['Idade_da_empresa'] = np.where(


14     4.0
23     6.0
35     9.0
56    20.0
68    11.0
Name: Idade_da_empresa, dtype: float64

### Filtrando os dados para que somente variaveis numéricas sejam utilizadas no R

In [23]:
# Checar quais colunas são numéricas
colunas_numericas = empresas_2012.select_dtypes(include=[np.number]).columns
colunas_nao_numericas = empresas_2012.select_dtypes(exclude=[np.number]).columns

# Exibir as colunas numéricas e não numéricas
print("Colunas numéricas:")
print(colunas_numericas)

print("\nColunas não numéricas:")
print(colunas_nao_numericas)

# Remover as colunas não numéricas do DataFrame
empresas_2012 = empresas_2012.select_dtypes(include=[np.number])

# Verificar o DataFrame resultante
print("\nDataFrame somente com colunas numéricas:")
print(empresas_2012)

Colunas numéricas:
Index(['comp_id', 'amort', 'curr_assets', 'curr_liab', 'extra_exp',
       'extra_inc', 'extra_profit_loss', 'fixed_assets', 'inc_bef_tax',
       'intang_assets', 'inventories', 'liq_assets', 'material_exp',
       'personnel_exp', 'profit_loss_year', 'sales', 'share_eq',
       'subscribed_cap', 'tang_assets', 'balsheet_flag', 'balsheet_length',
       'balsheet_notfullyear', 'year', 'nace_main', 'ind2', 'ind', 'urban_m',
       'founded_year', 'operates_within_2_years', 'Idade_da_empresa'],
      dtype='object')

Colunas não numéricas:
Index(['begin', 'end', 'region_m', 'founded_date'], dtype='object')

DataFrame somente com colunas numéricas:
             comp_id         amort   curr_assets     curr_liab  extra_exp  \
14      1.001541e+06    481.481476  9.629630e+03  1.303704e+03   0.000000   
23      1.002029e+06  14929.629883  2.038852e+05  1.204445e+05   0.000000   
35      1.003200e+06     25.925926  2.222222e+01  1.099630e+04   0.000000   
56      1.011889e+

### Checando colunas com números nulos:

In [24]:

# Calcular a porcentagem de valores zero em cada coluna
percentual_zero = (empresas_2012 == 0).mean() * 100

# Exibir a porcentagem de valores zero por coluna em ordem decrescente
percentual_zero_ordenado = percentual_zero.sort_values(ascending=False)

print("Porcentagem de valores zero por coluna (em ordem decrescente):")
print(percentual_zero_ordenado)

Porcentagem de valores zero por coluna (em ordem decrescente):
balsheet_flag              100.000000
balsheet_notfullyear        99.500988
extra_exp                   91.887352
extra_inc                   91.551383
extra_profit_loss           86.338933
intang_assets               84.723320
inventories                 35.810277
tang_assets                 23.176877
fixed_assets                22.223320
operates_within_2_years     19.708498
amort                       18.231225
personnel_exp                8.730237
profit_loss_year             3.241107
curr_liab                    1.744071
liq_assets                   1.576087
curr_assets                  0.424901
material_exp                 0.227273
Idade_da_empresa             0.182806
subscribed_cap               0.153162
inc_bef_tax                  0.148221
share_eq                     0.044466
ind                          0.000000
founded_year                 0.000000
urban_m                      0.000000
comp_id                  

In [25]:
# Definir o limite de porcentagem de valores zero aceitável
limite_zero = 50.0  # Exemplo: remover colunas com mais de 30% de valores zero

# Remover as colunas com porcentagem de valores zero acima do limite
df_limpo = empresas_2012.loc[:, percentual_zero <= limite_zero]

In [26]:
# Calcular a porcentagem de valores zero em cada coluna
percentual_zero = (df_limpo == 0).mean() * 100

# Exibir a porcentagem de valores zero por coluna em ordem decrescente
percentual_zero_ordenado = percentual_zero.sort_values(ascending=False)

print("Porcentagem de valores zero por coluna (em ordem decrescente):")
print(percentual_zero_ordenado)

Porcentagem de valores zero por coluna (em ordem decrescente):
inventories                35.810277
tang_assets                23.176877
fixed_assets               22.223320
operates_within_2_years    19.708498
amort                      18.231225
personnel_exp               8.730237
profit_loss_year            3.241107
curr_liab                   1.744071
liq_assets                  1.576087
curr_assets                 0.424901
material_exp                0.227273
Idade_da_empresa            0.182806
subscribed_cap              0.153162
inc_bef_tax                 0.148221
share_eq                    0.044466
balsheet_length             0.000000
year                        0.000000
nace_main                   0.000000
ind2                        0.000000
ind                         0.000000
urban_m                     0.000000
founded_year                0.000000
sales                       0.000000
comp_id                     0.000000
dtype: float64


In [27]:
colunas = ["balsheet_length", "year","comp_id","ind","ind2","urban_m"]
dadosR = df_limpo.drop(columns = colunas)
dadosR.head()

,amort,curr_assets,curr_liab,fixed_assets,inc_bef_tax,inventories,liq_assets,material_exp,personnel_exp,profit_loss_year,sales,share_eq,subscribed_cap,tang_assets,nace_main,founded_year,operates_within_2_years,Idade_da_empresa
14,481.481476,9629.629883,1303.703735,190566.671875,-7696.296387,0.000000,9048.148438,8351.851562,0.000000,-7722.222168,7.093989,1.912630e+05,200740.734375,190566.671875,5610.0,2008.0,1,4.0
23,14929.629883,203885.187500,120444.453125,23459.259766,11818.518555,677.777771,15077.777344,984270.375000,41037.035156,9722.222656,13.943478,9.314445e+04,11111.111328,23459.259766,2711.0,2006.0,1,6.0
35,25.925926,22.222221,10996.295898,0.000000,-2337.037109,0.000000,22.222221,1933.333374,1355.555542,-2340.740723,7.932429,-2.378519e+04,1851.851807,0.000000,5630.0,2003.0,0,9.0
56,36625.925781,160166.671875,18911.111328,933574.062500,96751.851562,11866.666992,131766.671875,204659.265625,92614.812500,96751.851562,12.980034,1.071011e+06,17481.482422,932037.062500,5510.0,1992.0,1,20.0
68,12551.851562,199903.703125,8274.074219,118229.632812,-1429.629639,0.000000,18585.185547,66744.445312,55711.109375,-2351.851807,11.773216,3.098852e+05,11111.111328,118229.632812,5510.0,2001.0,1,11.0


In [28]:
dadosR = dadosR.dropna()

count    18729.000000
mean        10.813548
std          1.692198
min          6.912448
25%          9.690081
50%         10.744517
75%         11.870063
max         16.110018
Name: sales, dtype: float64

## Criando o dataset para treinar o modelo

In [29]:
dadosR.to_csv("dados_para_o_R.csv", index = True)